In [ ]:
!pip install kaggle

from google.colab import files
#upload a file named "kaggle.json" given in repository
files.upload()

In [2]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
#download mask-detection dataset
!kaggle datasets download -d shanmukh05/mask-detection

In [4]:
#import necessary libraries
import matplotlib.pyplot as plt
import numpy as np
import os
import zipfile
import PIL
import PIL.Image
import tensorflow as tf
import pathlib
from tensorflow.keras import layers
from tensorflow.keras.optimizers import RMSprop
import datetime
from tensorboard.plugins.hparams import api as hp

In [ ]:
#path of downloaded zipfile (changes accordingly)
path = "/content/mask-detection.zip"
zipref = zipfile.ZipFile(path, "r")
zipref.extractall("/content/")
zipref.close()

data_dir = "/content/mask_detection/"
data_dir = pathlib.Path(data_dir)
image_count = len(list(data_dir.glob("*/*.jpg")))
print(image_count)
no = list(data_dir.glob("no/*"))
PIL.Image.open(str(no[6]))

In [ ]:
batch_size = 32
height = 300
width = 300

#prepare training data
train_data = tf.keras.preprocessing.image_dataset_from_directory(
    data_dir,
    validation_split = 0.1,
    subset = "training",
    seed =123,
    image_size =(height,width),
    batch_size = batch_size
)

#prepare validation data
val_data = tf.keras.preprocessing.image_dataset_from_directory(
    data_dir,
    validation_split = 0.1,
    subset = "validation",
    seed=123,
    image_size=(height,width),
    batch_size=batch_size
)

classes = train_data.class_names
print(classes)

plt.figure(figsize=(10, 10))
for images, labels in train_data.take(1):
  for i in range(9):
    ax = plt.subplot(3, 3, i + 1)
    plt.imshow(images[i].numpy().astype("uint8"))
    plt.title(classes[labels[i]])
    plt.axis("off")

images = images.numpy().astype("uint8")

In [7]:
#load tensorboard necessities
%load_ext tensorboard

In [8]:
#remove the log folder if any present
!rm -rf ./logs/

In [9]:
#visualizing training images using TensorBoard

logdir = "logs/train_data/"+datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
file_writer = tf.summary.create_file_writer(logdir)

with file_writer.as_default():
  tf.summary.image("Training data",images,max_outputs=32,step=0)

In [ ]:
#visualizing training images using TensorBoard

%tensorboard --logdir logs/train_data

In [21]:
#Hyperparametr tuning using TensorBoard

H_DROPOUT = hp.HParam("dropout",hp.RealInterval(0.2,0.5))
H_OPTIMIZER =   hp.HParam("optimizer",hp.Discrete(["adam"]))
METRIC = "accuracy" 

with tf.summary.create_file_writer("logs/hparam_tuning").as_default():
  hp.hparams_config(
      hparams = [H_DROPOUT,H_OPTIMIZER],
      metrics = [hp.Metric(METRIC,display_name="Accuracy")]
  )

In [13]:
#visualizing accuracy,weights(bias,kernel weights) in tensorboard

logdir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=logdir,histogram_freq=1)

In [22]:
#optimize the input of data while running
AUTOTUNE = tf.data.experimental.AUTOTUNE
train_data = train_data.cache().shuffle(1000).prefetch(buffer_size=AUTOTUNE)
val_data = val_data.cache().prefetch(buffer_size = AUTOTUNE)


#data augumentation
data_augmentation = tf.keras.Sequential(
  [
    layers.experimental.preprocessing.RandomFlip("horizontal",input_shape=(height, width,3)),
    layers.experimental.preprocessing.RandomRotation(0.1),
    layers.experimental.preprocessing.RandomZoom(0.1),
  ]
)

#define the model
def train_model(hparams):
  model = tf.keras.models.Sequential([
  data_augmentation,
  layers.experimental.preprocessing.Rescaling(1./255),
  layers.Conv2D(16, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(32, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(64, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Dropout(hparams[H_DROPOUT]),
  layers.Flatten(),
  layers.Dense(128, activation='relu'),
  layers.Dense(64, activation='relu'),
  layers.Dense(32, activation='relu'),
  layers.Dense(1,activation='sigmoid')
  ])

  model.compile(optimizer=hparams[H_OPTIMIZER],
              loss='binary_crossentropy',
              metrics=['accuracy'])
  epochs=20
  logdir = "logs/hparam_tuning"
  history=model.fit(
  train_data,
  validation_data=val_data,
  epochs=epochs,
  verbose=1)
  accuracy = history.history["val_accuracy"]
  return accuracy[epochs-1]

In [32]:
#For each run, log an hparams summary with the hyperparameters and final accuracy

def run(dir,hparams):
  with tf.summary.create_file_writer(dir).as_default():
    hp.hparams(hparams)
    accuracy = train_model(hparams)
    tf.summary.scalar(METRIC,accuracy,step=1)
  return accuracy

In [ ]:
#Start runs and log them all under one parent directory

session_num = 0
best_dropout = 0
best_optimizer = None
high_acc =0
for dropout_rate in (H_DROPOUT.domain.min_value, H_DROPOUT.domain.max_value):
  for optimizer in H_OPTIMIZER.domain.values:
      hparams = {
          H_DROPOUT: dropout_rate,
          H_OPTIMIZER: optimizer,
      }
      run_name = "run-%d" % session_num
      print('Starting trial: %s' % run_name)
      print({h.name: hparams[h] for h in hparams})
      accuracy=run('logs/hparam_tuning/' + run_name, hparams)
      if accuracy > high_acc:
        high_acc=accuracy
        print(high_acc)
        best_dropout = hparams[H_DROPOUT]
        best_optimizer = hparams[H_OPTIMIZER]
      session_num += 1

In [ ]:
%tensorboard --logdir logs/hparam_tuning

In [ ]:
#visualizing accuracy,weights(bias,kernel weights) in tensorboard
%tensorboard --logdir logs/fit

In [ ]:
#finding the best model
print("Best Dropout parameter: ",best_dropout)
print("Best optimizer: ",best_optimizer)
model = tf.keras.models.Sequential([
  data_augmentation,
  layers.experimental.preprocessing.Rescaling(1./255),
  layers.Conv2D(16, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(32, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(64, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Dropout(best_dropout),
  layers.Flatten(),
  layers.Dense(128, activation='relu'),
  layers.Dense(64, activation='relu'),
  layers.Dense(32, activation='relu'),
  layers.Dense(1,activation='sigmoid')
  ])

model.compile(optimizer=best_optimizer,
              loss='binary_crossentropy',
              metrics=['accuracy'])
epochs=20
  
history=model.fit(
  train_data,
  validation_data=val_data,
  epochs=epochs,
  verbose=1)

In [ ]:
#visualizing the accuracy and loss of train and validation dataset 
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs_range = range(epochs)
plt.figure(figsize=(8, 8))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')
plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()

In [ ]:
#predict for unseen multiple images
ls = os.listdir("/content/prediction/")
path = "/content/prediction/"
#ls.remove('.ipynb_checkpoints')
img_arr=[]
predictions =[]
scores =[]
for j,i in enumerate(ls):
  img = tf.keras.preprocessing.image.load_img(
      path+i,target_size = (height,width)
  )
  arr = tf.keras.preprocessing.image.img_to_array(img)
  img_arr.append(arr)
  arr = tf.expand_dims(arr,0)
  pred = model.predict(arr)
  prediction=pred[0][0]
  if prediction <0.5:
    predictions.append(0)
    scores.append(100*(1-prediction))
  else:
    predictions.append(1)
    scores.append(100*prediction)
scores = [round(i,2) for i in scores]
txt = ["didn't wear mask ","wore mask "]
plt.figure(figsize=(20,20))
for i in range(25):
  ax = plt.subplot(5,5, i + 1)
  plt.imshow(img_arr[i].astype("uint8"))
  text = str(txt[predictions[i]])+"("+str(scores[i])+"% acc)"
  plt.title(text)
  plt.axis("off")


In [ ]:
#for single unseen image
path = "/content/prediction/11.jpg"

img = tf.keras.preprocessing.image.load_img(
    path,target_size = (height,width)
)

img_arr = tf.keras.preprocessing.image.img_to_array(img)
img_arr = tf.expand_dims(img_arr,0)
prediction = model.predict(img_arr)
print(prediction[0][0])
score = prediction[0]
if score<0.5:
  print("The person in the given image didn't wear a mask")
else:
  print("The person in the image wore mask")

PIL.Image.open(str(path))

In [ ]:
print(model.summary())

In [ ]:
#outputs of different layers for training data
layer1= model.get_layer("rescaling_17")
layer2 = model.get_layer("conv2d_51")
layer3 = model.get_layer("max_pooling2d_51")
layer4 = model.get_layer("conv2d_52")
plt.figure(figsize=(10, 10))
for img,label in train_data.take(1):
  out1 = layer1(img)
  out2 = layer2(out1)
  out3= layer3(out2)
  out4 = layer4(out3)
  for i in range(1):
    img_arr = tf.keras.preprocessing.image.img_to_array(img[i])
    img_arr = tf.expand_dims(img_arr,0)
    pred = model.predict(img_arr)*100
    pred =round(pred[0][0],2)
    ax = plt.subplot(3,4,1+3*i)
    plt.imshow(img[i].numpy().astype("uint8"))
    plt.title(str(pred))
    ax = plt.subplot(3,4,2+3*i)
    im1 = np.sum(out2[i],axis=2)
    plt.imshow(im1.astype("uint8"))
    plt.title(str(pred))
    ax = plt.subplot(3,4,3+3*i)
    im2 = np.sum(out3[i],axis=2)
    plt.imshow(im2.astype("uint8"))
    plt.title(str(pred))
    ax = plt.subplot(3,4,4+3*i)
    im3 = np.sum(out4[i],axis=2)
    plt.imshow(im3.astype("uint8"))
    plt.title(str(pred))

In [ ]:
#outputs of different layers for unseen images
path = "/content/prediction/25.jpg"
img = tf.keras.preprocessing.image.load_img(
    path,target_size = (height,width)
)
img_arr = tf.keras.preprocessing.image.img_to_array(img)
img_arr = tf.expand_dims(img_arr,0)
pred = model.predict(img_arr)*100
pred =round(pred[0][0],2)

#names of the layers will be changed as per model.summary() output

layer1= model.get_layer("rescaling_17")
layer2 = model.get_layer("conv2d_51")
layer3 = model.get_layer("max_pooling2d_51")
layer4 = model.get_layer("conv2d_52")
layer5 = model.get_layer("max_pooling2d_52")
layer6 = model.get_layer("conv2d_53")
layer7 = model.get_layer("max_pooling2d_53")
layer8 = model.get_layer("dropout_17")

out1 = layer1(img_arr)
out2 = layer2(out1)
out3 = layer3(out2)
out4 = layer4(out3)
out5 = layer5(out4)
out6 = layer6(out5)
out7 = layer7(out6)
out8 = layer8(out7)
num =5
plt.figure(figsize=(30,30))
ax = plt.subplot(1,num,1)
plt.imshow(img_arr[0].numpy().astype("uint8"))
plt.title(str(pred))
ax = plt.subplot(1,num,2)
plt.imshow(np.sum(out2[0],axis=2).astype("uint8"))
plt.title(str(pred))
ax = plt.subplot(1,num,3)
plt.imshow(np.sum(out3[0],axis=2).astype("uint8"))
plt.title(str(pred))
ax = plt.subplot(1,num,4)
plt.imshow(np.sum(out6[0],axis=2).astype("uint8"))
plt.title(str(pred))
ax = plt.subplot(1,num,5)
plt.imshow(np.sum(out8[0],axis=2).astype("uint8"))
plt.title(str(pred))

In [ ]:
#outputs of various filter in given convolution layer
img_arr = out6[0].numpy()
x,y,z = img_arr.shape
arr = img_arr.reshape(x*y*z,1)*255
plt.figure(figsize=(20,20))
for i in range(16):
  ax = plt.subplot(5,4,i+1)
  plt.imshow(arr[x*y*i:x*y*i+x*y].reshape(x,y).astype("uint8"))
  plt.title(str(pred))
ax = plt.subplot(5,4,17)
plt.imshow(np.sum(img_arr,axis=2).astype("uint8"))